In [ ]:
from bs4 import BeautifulSoup
import requests
from IPython.core.display import display, HTML
import pandas as pd
import numpy as np
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from fake_useragent import UserAgent
from selenium.webdriver.chrome.options import Options
import random

# Web Scraping: Selenium

In [ ]:
options = Options()
ua = UserAgent()
user_agent = ua.random
print(user_agent)
options.add_argument(user_agent)
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver


## Collect Company name- Method 1 from company review page

In [ ]:
query = "Data"
indeed_search = "https://www.indeed.com/companies/search?q="
indeed_query = indeed_search + query + "&l=&from=discovery-cmp-search"
indeed_query


In [ ]:
from fake_useragent import UserAgent
from selenium.webdriver.chrome.options import Options
options = Options()
ua = UserAgent()
user_agent = ua.random
print(user_agent)
options.add_argument(f'user-agent={user_agent}')

In [ ]:
driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
driver.get(indeed_query)

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
cmp_n=soup.find_all('a',{"itemprop":"name","class":"cmp-CompanyWidget-name"})

#for cmp in cmp_n[1:]:
#    print(cmp.text)

In [ ]:
job_title=["Data+Analyst","Data+Engineers","Database+Administrator",
           "Machine+Learning+Engineer","Data+Scientist","Data+Architect",
           "Statistician""Business+Analyst","Data","Analytics+Manager",
           "Big+Data+Engineer","Database+Developer","Statistician",
           "Big+Data+Software+Developer","Business+Analyst",
           "Business+Intelligence+Analyst","Analyst","Analysis","Python","SQL",
           "BI+Specialist","Analytics+Manager","Machine+Learning+Scientist","Big+Data"]

In [ ]:
def web_scrapping_cmp(title):
    
    headers = ['company']
    query = title
    indeed_search = "https://www.indeed.com/companies/search?q="
    indeed_query = indeed_search + query + "&l=&from=discovery-cmp-search"

    options = Options()
    ua = UserAgent()
    user_agent = ua.random
    options.add_argument(f'user-agent={user_agent}')

    driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
    driver.get(indeed_query)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    
    cmp_n=soup.find_all('a',{"itemprop":"name","class":"cmp-CompanyWidget-name"})
    company_name=[]
    for cmp in cmp_n[1:]:
        company_name.append(cmp.text)
    headers = ['company']*len(company_name)
    driver.quit()
    return dict(zip(headers, [company_name]))

In [ ]:
company_list = []

for link in job_title:
    company_list.append(web_scrapping_cmp(link))

In [ ]:
print(len(company_list)) #23
#company_list

In [ ]:
Indeed_Company=pd.DataFrame(company_list[0])
for i in range(1,len(company_list)):
    try:
        Indeed_Company=pd.concat([Indeed_Company,pd.DataFrame(company_list[i])])
    except Exception:
        pass

In [ ]:
#driver.quit()
len(Indeed_Company.company.unique())
cmps=pd.DataFrame(Indeed_Company.company.unique())
cmps[0]

## Collect Company name- Method 2 from job searching page

In [ ]:
query_1 = "data-scientist"
indeed_search = "https://www.indeed.com/q-"
indeed_query_1 = indeed_search + query_1 + "-jobs.html"
indeed_query_1


In [ ]:
driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
driver.get(indeed_query_1)
next_page=driver.find_element_by_class_name('pn')
next_page.click()
#next_page=driver.find_element_by_class_name('pn')
#next_page.click()

next_page=driver.find_elements_by_css_selector('span.np')
next_page[1].click()
#<path d="M10 6L8.59 7.41 13.17 12l-4.58 4.59L10 18l6-6-6-6z" fill="#2D2D2D"></path>

In [ ]:
data_title=["data-scientist","data-analyst","Data-Engineers","Data-Architect","Database-Developer"
            ,"Machine-Learning-Scientist","Machine-Learning-Engineer"]

#Indeed_Company=[]
for title in data_title:
    query_1 = title
    indeed_search = "https://www.indeed.com/q-"
    indeed_query_1 = indeed_search + query_1 + "-jobs.html"
    driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
    driver.get(indeed_query_1)
    
    for i in range(35):
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        comp_name=soup.find_all('a', {"data-tn-element":"companyName"})

        for cmp in comp_name:
            Indeed_Company.append(cmp.text.replace("\n",""))
            
        time.sleep(2)
        
        if i==0:
            next_page=driver.find_element_by_class_name('np')
            next_page.click()
        else:
            try:
                next_page=driver.find_elements_by_css_selector('span.np')
                next_page[1].click()
            except Exception:
                pass
    driver.quit()
    time.sleep(300)

print(len(Indeed_Company))


In [ ]:
#driver.quit()
print(len(pd.DataFrame(Indeed_Company)[0].unique()))
new_company_list= pd.concat([cmps,pd.DataFrame(Indeed_Company)])
new_company_list=pd.DataFrame(new_company_list[0].unique())
new_company_list.to_csv('new_company_list.csv', index=False)

### 1. Salary page

In [ ]:
query1 = "Paypal/salaries"
indeed_search = "https://www.indeed.com/cmp/"
indeed_query1 = indeed_search + query1 +"?job_category=math"
indeed_query1

In [ ]:
from fake_useragent import UserAgent
from selenium.webdriver.chrome.options import Options
options = Options()
ua = UserAgent()
user_agent = ua.random
options.add_argument(f'user-agent={user_agent}')

##### First we launch the indeed page through our ChromeDrive. 
(A new browser should pop up. To continue using Selenium, keep this window open!)

In [ ]:
driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
driver.get(indeed_query1)

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

#### Get Title/Salary/hourly or year

In [ ]:
a=soup.find_all('span')

for a1 in a:
    try:
        a2=a1.text.split(" ")
        if a2[1]=='per':
            print(a2[0], a2[1]+" "+a2[2])
    except Exception:
        pass


In [ ]:
soup.find_all('div', class_="cmp-SalarySummaryTitle")[0].find('a').text


In [ ]:
salary=soup.find_all('span')
title=soup.find_all('div', class_="cmp-SalarySummaryTitle")

i=0
for salary1 in salary:
    try:
        salary2=salary1.text.split(" ")
        if salary2[1]=='per':
            print(salary2[0], salary2[1]+" "+salary2[2])
            print(title[i].find('a').text)
            i+=1
    except Exception:
        pass

print(i)


#### Salary Satisfication rate %

In [ ]:
#salary_sat=soup.find('div',class_="cmp-SalarySatisfactionSidebarWidgetPieChart-inside").text
salary_sat=soup.find('div',class_="cmp-SalarySatisfactionSidebarWidgetPieChart-inside").text
salary_sat=int(salary_sat.replace("%",""))
salary_sat

In [ ]:
#### Company Rating value & benefit rate

In [ ]:
cmp_rating=float(soup.find('span', {"aria-hidden":"true", "class":"cmp-CompactHeaderCompanyRatings-value"}).text)
cmp_rating                 

In [ ]:
query1 = "Paypal"
indeed_search = "https://www.indeed.com/cmp/"
indeed_query1 = indeed_search + query1

#driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
#driver.get(indeed_query1)
#soup = BeautifulSoup(driver.page_source, 'html.parser')

benefit_rating=float(soup.find_all('span', {"aria-hidden":"true","class":"css-3xwreb e1wnkr790"})[1].text)
print(benefit_rating)



#### 2. Company home page

In [ ]:
query_2 = "Paypal"
indeed_search_2 = "https://www.indeed.com/cmp/"
indeed_query_2 = indeed_search_2 + query_2 + "/about?from=acme-wonder"
print(indeed_query_2)
driver.get(indeed_query_2)

In [ ]:
soup_2 = BeautifulSoup(driver.page_source, 'html.parser')


#### Get Headquarters/Revenue/company size/ Industry

In [ ]:
Headquarters_State=soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content",
                   "data-testid":"headquarters"}).text.split(',')[1]

Headquarters_City=soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content",
                   "data-testid":"headquarters"}).text.split(',')[0]

#Headquarters_Country=soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content",
#                   "data-testid":"headquarters"}).text.split(',')[2]

Revenue= soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content", "data-testid":"revenue"}).text


Cmp_size=soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content","data-testid":"employees"}).text

Industry= soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content","data-testid":"revenue"}).text

print(Headquarters_State,Headquarters_City, Cmp_size,Revenue,Industry)

### 3. Company Interview

In [ ]:
query_3 = "Paypal/interviews"
indeed_search_3 = "https://www.indeed.com/cmp/"
indeed_query_3 = indeed_search_3 + query_3
print(indeed_query_3)
driver.get(indeed_query_3)

In [ ]:
soup_3 = BeautifulSoup(driver.page_source, 'html.parser')

##### Get Interview difficulty & length

In [ ]:
inter_diff=soup_3.find('div',{"class":"cmp-DifficultyCard-result",
                       "data-testid":"cmp-InterviewSummaryCard-result"}).text
print(inter_diff)

In [ ]:
inter_len=soup_3.find('div',{"class":"cmp-DurationCard-text",
                       "data-testid":"cmp-InterviewSummaryCard-result"}).text
print(inter_len)


# Create Function to web scrapping

In [ ]:
def web_scrapping(cmp):
    
    headers = ['company','Headquarters_State', 'Headquarters_City', 'revenue', 
               'cmp_size', 'industry','salary_sat', 'inter_diff','inter_len','title', 'salary', 'salary_unit',
              "cmp_rating","benefit_rating"]
    
    cmp1=cmp.replace(" ", "-")
    
    try:
        query1 = cmp1+"/salaries"
        indeed_search = "https://www.indeed.com/cmp/"
        indeed_query1 = indeed_search + query1 +"?job_category=math"
        driver.get(indeed_query1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        ###Get Title/Salary/hourly or year
        salary_span=soup.find_all('span')
        title1=soup.find_all('div', class_="cmp-SalarySummaryTitle")
        
        salary=[]
        salary_unit=[]
        title=[]
        
        i=0
        for salary1 in salary_span:
            try:
                salary2=salary1.text.split(" ")
                if salary2[1]=='per':
                    salary.append(salary2[0])
                    salary_unit.append(salary2[1]+" "+salary2[2])
                    title.append(title1[i].find('a').text)
                    i+=1
            except Exception:
                pass
        ###Get Salary Satisfication rate %
        salary_sat=soup.find('div',class_="cmp-SalarySatisfactionSidebarWidgetPieChart-inside").text
        salary_sat_div=int(salary_sat.replace("%",""))
        salary_sat=[salary_sat_div]*len(salary)
        cmp_rating=[float(soup.find('span', {"aria-hidden":"true", 
                                             "class":"cmp-CompactHeaderCompanyRatings-value"}).text)]*len(salary)
        
        
        query1 = cmp1
        indeed_search = "https://www.indeed.com/cmp/"
        indeed_query1 = indeed_search + query1
        driver.get(indeed_query1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        benefit_rating=[float(soup.find_all('span', {"aria-hidden":"true","class":"css-3xwreb e1wnkr790"})[1].text)]*len(salary)

        ###Get Headquarters/Revenue/company size/ Industry
        query_2 = cmp1
        indeed_search_2 = "https://www.indeed.com/cmp/"
        indeed_query_2 = indeed_search_2 + query_2 + "/about?from=acme-wonder"
        driver.get(indeed_query_2)
        soup_2 = BeautifulSoup(driver.page_source, 'html.parser')
        
 
        try:     
            Headquarters_State=[soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content",
                                                  "data-testid":"headquarters"}).text.split(',')[1]]*len(salary)
        except Exception:
            Headquarters_State=np.nan*len(salary)

        try:
            Headquarters_City=[soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content",
                                                 "data-testid":"headquarters"}).text.split(',')[0]]*len(salary)
        except Exception:
            Headquarters_City=np.nan*len(salary)
            
        try:    
            revenue= [soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content", 
                                        "data-testid":"revenue"}).text]*len(salary)
        except Exception:
            revenue=np.nan*len(salary) 
            
        try:
            cmp_size=[soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content",
                                         "data-testid":"employees"}).text]*len(salary)
        except Exception:
            cmp_size=np.nan*len(salary) 
            
        try:
            industry= [soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content",
                                          "data-testid":"revenue"}).text]*len(salary)
        except Exception:
            industry=np.nan*len(salary) 
    
        ###Get Interview difficulty & length
        query_3 = cmp1 + "/interviews"
        indeed_search_3 = "https://www.indeed.com/cmp/"
        indeed_query_3 = indeed_search_3 + query_3
        driver.get(indeed_query_3)
        soup_3 = BeautifulSoup(driver.page_source, 'html.parser')
        try:
            inter_diff=[soup_3.find('div',{"class":"cmp-DifficultyCard-result",
                           "data-testid":"cmp-InterviewSummaryCard-result"}).text]*len(salary)
        except Exception:
            inter_diff="none"*len(salary)
            
        try:
            inter_len=[soup_3.find('div',{"class":"cmp-DurationCard-text",
                           "data-testid":"cmp-InterviewSummaryCard-result"}).text]*len(salary)
        except Exception:
            inter_len="none"*len(salary)
        
        
        company=[cmp]*len(salary)
        return dict(zip(headers, [company,Headquarters_State, Headquarters_City, revenue, 
               cmp_size, industry,salary_sat, inter_diff,inter_len,title, salary, salary_unit,cmp_rating,benefit_rating]))


    except Exception:
        return dict(zip(headers, [cmp, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0,0]))

### For loop-web_scrapping function

In [ ]:
#indeed_data = []
change_user=0
sleep_time=0

for link in new_company_list[0][3:]:

    if change_user==0:
        options = Options()
        ua = UserAgent()
        user_agent = ua.random
        options.add_argument(f'user-agent={user_agent}')
        driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)

    else:
        driver.quit()
        options = Options()
        ua = UserAgent()
        user_agent = ua.random
        options.add_argument(f'user-agent={user_agent}')
        driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)

    change_user+=1
    sleep_time+=1
 
    indeed_data.append(web_scrapping(link))
    if sleep_time % 10==0:
        time.sleep(30+5*random.random())
        
    if sleep_time % 100==0:
        time.sleep(300)
driver.quit()


In [ ]:
#indeed_data_1=[]
change_user=1
sleep_time=1

for link in issue_cmp[3:]:

    if change_user==0:
        options = Options()
        ua = UserAgent()
        user_agent = ua.random
        options.add_argument(f'user-agent={user_agent}')
        driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)

    else:
        driver.quit()
        options = Options()
        ua = UserAgent()
        user_agent = ua.random
        options.add_argument(f'user-agent={user_agent}')
        driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)

    change_user+=1
    sleep_time+=1
 
    indeed_data_1.append(web_scrapping1(link))
    if sleep_time % 50==0:
        time.sleep(30+5*random.random())
        
    if sleep_time % 200==0:
        time.sleep(300)
driver.quit()

In [ ]:
len(indeed_data_1)
c=pd.DataFrame(indeed_data_1[0])
for i in range(1,len(indeed_data_1)):
    try:
        Indeed_Company_Salary_1=pd.concat([Indeed_Company_Salary_1,pd.DataFrame(indeed_data_1[i])])
    except Exception:
        pass

In [ ]:
#company_list.remove_all(None)
#print(company_list)

Indeed_Company_Salary=pd.DataFrame(indeed_data[0])
for i in range(1,len(indeed_data)):
    try:
        Indeed_Company_Salary=pd.concat([Indeed_Company_Salary,pd.DataFrame(indeed_data[i])])
    except Exception:
        pass

In [ ]:
print(Indeed_Company_Salary.shape)

Indeed_Company_Salary.head(10)

In [ ]:
Indeed_Company_Salary.to_csv('Indeed_Company_Salary.csv', index=False)

In [ ]:
driver.quit()

In [ ]:
#Indeed_Company_Salary=pd.read_csv("Indeed_Company_Salary_revised.csv")

#Indeed_Company_Salary=pd.concat([Indeed_Company_Salary,Indeed_Company_Salary_1])

#Indeed_Company_Salary.to_csv('Indeed_Company_Salary.csv', index=False)

### Check the company without data, and rerun webscrapping

In [ ]:
print(len(indeed_data))
empty=[]
for i,a in enumerate(indeed_data):
    if a==({'company': [],'Headquarters_State': [],
                   'Headquarters_City': [],'revenue': [],'cmp_size': [],'industry': [],
                   'salary_sat': [],'inter_diff': [],'inter_len': [],'title': [],'salary': [],
                   'salary_unit': [],'cmp_rating': [],'benefit_rating': []}):
        empty.append(i)

empty=np.array(empty)+1    
empty

issue_cmp=new_company_list[0][empty]
len(issue_cmp)
print(issue_cmp)


#indeed_data.index({'company': [],'Headquarters_State': [],
#                   'Headquarters_City': [],'revenue': [],'cmp_size': [],'industry': [],
#                   'salary_sat': [],'inter_diff': [],'inter_len': [],'title': [],'salary': [],
#                   'salary_unit': [],'cmp_rating': [],'benefit_rating': []})
#for i in range(13):
#    print(indeed_data[i]['company'])
#new_company_list[0][13]
#indeed_data[3]

### Founded year & industry

In [ ]:
company=Indeed_Company_Salary['company'].unique()

In [ ]:
driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
query=company[1].replace(" ","-")
indeed_search = "https://www.indeed.com/cmp/"
indeed_query = indeed_search + query
driver.get(indeed_query)
soup = BeautifulSoup(driver.page_source, 'html.parser')


In [ ]:
def found_industry(cmp):
    try:
        header=["company","founded","industry"]
        company=cmp

        query=cmp.replace(" ","-")
        indeed_search = "https://www.indeed.com/cmp/"
        indeed_query = indeed_search + query
        driver.get(indeed_query)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        founded=soup.find('li', {"data-testid":"companyInfo-founded", "class":"css-v4e08k eu4oa1w0"}).text
        founded=int(founded.replace("Founded",""))

        industry=soup.find('li', {"data-testid":"companyInfo-industry", "class":"css-v4e08k eu4oa1w0"}).text
        industry=industry.replace("Industry","")


        return dict(zip(header, [company,founded, industry]))


    except Exception:
        return dict(zip(headers, [cmp, 0, 0]))

In [ ]:
change_user=0
sleep_time=0
indeed_data_2=[]

for link in company[0:]:

    if change_user==0:
        options = Options()
        ua = UserAgent()
        user_agent = ua.random
        options.add_argument(f'user-agent={user_agent}')
        driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)

    else:
        driver.quit()
        options = Options()
        ua = UserAgent()
        user_agent = ua.random
        options.add_argument(f'user-agent={user_agent}')
        driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)

    change_user+=1
    sleep_time+=1
 
    indeed_data_2.append(found_industry(link))
    if sleep_time % 50==0:
        time.sleep(20+5*random.random())
        
    if sleep_time % 200==0:
        time.sleep(300)
driver.quit()



In [ ]:
indeed_data_2[0]

In [ ]:



industry_found=pd.DataFrame(indeed_data_2[0],index=[0])

for i in range(1,len(indeed_data_2)):
    try:
        industry_found=pd.concat([industry_found,pd.DataFrame(indeed_data_2[i],index=[i])])
    except Exception:
        pass

industry_found.to_csv('industry_found.csv', index=False)

In [ ]:
industry_found

In [ ]:
pd.concat([industry_found,pd.DataFrame(indeed_data_2[1],index=[1])])